# Notebook 2: Going Deeper

**TurboPanda** has a lot of interesting functionality to do with manipulating columns and groups of columns that `pandas` so often lacks.

In [1]:
import sys
import numpy as np
import pandas as pd
sys.path.insert(0,"../")
# our main import
import turbopanda as turb

## Reading in our dataset

In [2]:
g = turb.read("../data/translation.csv", name="Translation")
g

../turbopanda/_deprecator.py:74: FutureWarning: Parameter(s) ['mode'] deprecated since version 0.2.1, to be removed in version 0.2.4; `delay` state no longer needed with compute()
  warnings.warn("".join(segments), FutureWarning)


MetaPanda(Translation(n=5216, p=14, mem=1.169MB, options=[]))

## Accessing with `head`

Normally copying over features from `pandas.DataFrame` was a forbidden fruit, but we decided in this case that `head` so useful that we would break tradition:

In [3]:
g.head(3)

colnames,prot_IDs,prot_names,Gene_names,translation_G1_1,translation_G1_2,translation_G1_3,translation_G2M_1,translation_G2M_2,translation_G2M_3,translation_MG1_1,translation_MG1_2,translation_MG1_3,translation_S_1,translation_S_2
counter,,,,,,,,,,,,,,
0,Q96IC2;Q96IC2-2;H3BM72;H3BV93;H3BSC5,Putative RNA exonuclease NEF-sp,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662
1,H0YGH4;P01023;H0YGH6;F8W7L3,Alpha-2-macroglobulin,A2M,22.62015,22.26825,23.11786,24.94606,24.21645,25.26399,23.56139,23.46051,22.21951,22.87688,23.35703
2,A8K2U0;F5H2W3;H0YGG5;F5H2Z2;F5GXP1,Alpha-2-macroglobulin-like protein 1,A2ML1,NaN,NaN,NaN,NaN,NaN,25.11629,NaN,NaN,NaN,NaN,NaN


## Generic `apply` to the underlying DataFrame

For the thousands of instances where we simply want to apply a `pandas.DataFrame.*` function to the underlying dataset, but retain the consistency between `df_` and `meta_` attributes in particular, our solution is to provide an `apply` function which is performed on the entire
dataset.

This is similar to the `transform` function we have, however `apply` only looks for functions within `pandas.DataFrame.*` API, and it does not allow for pre-subset selection (using the `selector` parameter) beforehand.

For instance, let's say we want to fill all the `NaN` values in dataset with a value. We could do this easily with `pandas.DataFrame.fillna` but to use `MetaPanda` we'd have to call the `transform` function with a custom lambda etc.

In [4]:
g.apply("fillna", 0)

MetaPanda(Translation(n=5216, p=14, mem=1.169MB, options=[]))

It's that easy.

`apply` also works on any basic `.str` accessor methods in addition to ones found in `pandas.DataFrame.*` API.

In [5]:
g.head(3)

colnames,prot_IDs,prot_names,Gene_names,translation_G1_1,translation_G1_2,translation_G1_3,translation_G2M_1,translation_G2M_2,translation_G2M_3,translation_MG1_1,translation_MG1_2,translation_MG1_3,translation_S_1,translation_S_2
counter,,,,,,,,,,,,,,
0,Q96IC2;Q96IC2-2;H3BM72;H3BV93;H3BSC5,Putative RNA exonuclease NEF-sp,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662
1,H0YGH4;P01023;H0YGH6;F8W7L3,Alpha-2-macroglobulin,A2M,22.62015,22.26825,23.11786,24.94606,24.21645,25.26399,23.56139,23.46051,22.21951,22.87688,23.35703
2,A8K2U0;F5H2W3;H0YGG5;F5H2Z2;F5GXP1,Alpha-2-macroglobulin-like protein 1,A2ML1,0.00000,0.00000,0.00000,0.00000,0.00000,25.11629,0.00000,0.00000,0.00000,0.00000,0.00000


## String manipulation of ID columns

Often datasets can have numerical/quantitative data coupled with IDs that help to identify rows based on your problem domain. In this case, we have some identifiers from **Uniprot**, one of the main databases managing protein sequences.

There is a slight problem though, each of these expression values has a *stack of protein IDs* associated with it. So we're going to have to untangle this somehow by **expanding** the ID columns to something we can then perform **set theory** operations on.

`MetaPanda` comes with an `expand` function which does exactly this:

In [6]:
g.expand("prot_IDs", sep=";")

MetaPanda(Translation(n=26318, p=14, mem=6.316MB, options=[]))

We can see in this instance that $n$ has increased significantly, as has the memory usage.

In [9]:
g.head(2)

colnames,prot_IDs,prot_names,Gene_names,translation_G1_1,translation_G1_2,translation_G1_3,translation_G2M_1,translation_G2M_2,translation_G2M_3,translation_MG1_1,translation_MG1_2,translation_MG1_3,translation_S_1,translation_S_2
counter,,,,,,,,,,,,,,
0,Q96IC2,Putative RNA exonuclease NEF-sp,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662
0,Q96IC2-2,Putative RNA exonuclease NEF-sp,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662


## Applying a transformation to eliminate duplicates

Here we see that there are labels that are *sort of* duplicated in the sense that they are appended with `-[0-9]`. We will split these off and keep the left-hand part, then apply a function which drops duplicates.

In [10]:
g.transform(lambda x: x.str.split("-",expand=True)[0], "prot_IDs")

MetaPanda(Translation(n=26318, p=14, mem=6.316MB, options=[]))

Now use `pandas.DataFrame.drop_duplicates`, with the subset just on Protein IDs

In [11]:
g.apply("drop_duplicates", subset=["prot_IDs"])

MetaPanda(Translation(n=21050, p=14, mem=5.052MB, options=[]))

In [12]:
g.head(3)

colnames,prot_IDs,prot_names,Gene_names,translation_G1_1,translation_G1_2,translation_G1_3,translation_G2M_1,translation_G2M_2,translation_G2M_3,translation_MG1_1,translation_MG1_2,translation_MG1_3,translation_S_1,translation_S_2
counter,,,,,,,,,,,,,,
0,Q96IC2,Putative RNA exonuclease NEF-sp,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662
0,H3BM72,Putative RNA exonuclease NEF-sp,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662
0,H3BV93,Putative RNA exonuclease NEF-sp,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662


## Using `mode=delay`: Delaying actions to make a task graph

**NOW DEPRECATED from version 0.2.4 onwards**

You may have noticed that the representation of the object shows `mode='instant'`.

Using the current approach, all of our operations apply instantly and inplace. This can be nice but if you make a mistake, it can be difficult to backtrack.

Fortunately, `MetaPanda` has a `mode` parameter during initialization and `mode_` attribute that can be set at any point. When set to `delay`, many functions that apply changes immediately to the data are instead *cached* in a `pipe_` attribute.

This attribute is then emptied when a call to `compute()` is made, and all of the operations within the `pipe_` are executed with their parameters, in order.

### Creating a series of operations to perform...

In [11]:
g.mode_="delay"

In [12]:
g.drop("prot_name")
g.apply("groupby", ["counter","Gene_names"])
g.apply("mean")
g.rename([("translation","trans")])

We can see that values have been entered into a `pipe_` attribute instead, with no changes to the underlying dataset.

## Pipelines

Pipes in TurboPanda allow you to chain together operations which are performed on the raw dataset and potentially the metadata information also. 

Pipe elements come in the following format:

    (<function name in MetaPanda, str>, <function arguments, tuple>, <function keyword arguments, dict>) 

For example if we wish to drop all columns containing `prot_name`:

```python
('drop', ('prot_name',), {})
```

Alternatively the *clean* version allows us just to pass the arguments as a list, with keywords as strings including an `=`:

```python
('drop', 'prot_name')
```



In [14]:
g.pipe_

{'current': []}

### Defining a pipe

We have a `Pipe` class for defining pipes:

In [17]:
turb.Pipe()

Pipe(n_elements=0)

### Adding pipe steps

Pipe operations can be chained together one after another, assuming each step returns a `pandas.DataFrame` of some kind, the main exception to this is `pandas.DataFrame.groupby`.

But if we check the `pipe_` attribute, all of the operations have been saved:

In [21]:
turb.Pipe(['drop', 'prot_name']).p

(('drop', ('prot_name',), {}),)

### Using `compute()`

By default, `compute()` accepts `pipe` and `inplace` as optional arguments, if `pipe is None` then it will automatically execute whatever is in the `pipe_` attribute and empty it. If `inplace` is set to False, a copy of the MetaPanda is returned instead of acting inplace.

Note that by default `inplace=False`, so it needs to be set to `True` to immediately update.

There is also a `multi_compute()` function which joins together different pipes before using regular `compute()`.

In [14]:
g.compute(inplace=True)

MetaPanda(Translation(n=5201, p=11, mem=0.562MB, key='None'), mode='instant')

### Functions that are affected by `delay` (accessed in `turb.metapanda.__delay_functions__`) include:


In [15]:
turb.metapanda.__delay_functions__

['drop',
 'keep',
 'apply',
 'apply_index',
 'apply_columns',
 'transform',
 'multi_transform',
 'rename',
 'add_prefix',
 'add_suffix',
 'meta_map',
 'expand',
 'shrink',
 'sort_columns',
 'split_categories',
 'melt',
 'filter_rows']

#### Non delay functions - always instant

In [16]:
turb.metapanda.__nondelay_functions__

['head',
 'cache',
 'multi_cache',
 'view',
 'view_not',
 'analyze',
 'compute',
 'multi_compute',
 'write',
 'write_json']

## Computing external pipelines

Often we want to have standardized pipelines that we can apply to many different `pandas.DataFrames`, for example we may have a similar system for **standardizing a dataset** in preparation of Machine Learning algorithms.

`turbopanda` provides one of these presets, called `ml_pipe`:

In [17]:
turb.pipes.ml_regression_pipe

<function turbopanda.pipes.ml_regression_pipe(mp, x_s, y_s, preprocessor='scale')>

This function takes a `MetaPanda`, with the input columns as a selector and output column(s) as a selector, with optional arguments, and returns a pipeline list which can be passed directly into `compute()`:

In [18]:
npipe = turb.ml_regression_pipe(g, "trans_G1_[1-3]", "trans_G2M_[1-3]")
npipe

[('drop', (object, '_id$', '_ID$', '^counter'), {}),
 ('apply', ('drop', Index([], dtype='object', name='colnames')), {'axis': 1}),
 ('apply',
  ('dropna',),
  {'axis': 0,
   'subset': Index(['trans_G2M_1', 'trans_G2M_2', 'trans_G2M_3'], dtype='object', name='colnames')}),
 ('transform',
  (<function turbopanda.pipes.ml_regression_pipe.<locals>.<lambda>(x)>,
   'trans_G1_[1-3]'),
  {}),
 ('transform',
  (<function sklearn.preprocessing._data.scale(X, axis=0, with_mean=True, with_std=True, copy=True)>,),
  {'selector': 'trans_G1_[1-3]', 'whole': True})]

By default `compute(inplace=False)` so a copy is returned.

In [19]:
ng = g.compute(npipe)
ng

MetaPanda(Translation(n=5201, p=11, mem=0.726MB, key='None'), mode='instant')

## Creating your own pipeline

Creating your own pipelines is incredibly useful for performing coherent stage-wise actions on a DataFrame, such as cleaning all of the column names, or preparing a dataset for Machine-Learning applications, or doing extensive preprocessing on a subset of columns.

Remember the three key parts of a pipeline argument:

    (<function name>, <function args>, <function kwargs>)

In [20]:
(
    # operation one, lower strings in df.columns
    ("apply_columns", ("lower",), {}),
    # operation two, lower strings in df.index, axis is redundant
    ("apply_index", ("lower",), {"axis": 0})
)

(('apply_columns', ('lower',), {}), ('apply_index', ('lower',), {'axis': 0}))

Instead of writing out all of that code, there is a relative shorthand using `turb.pipe`.

This allows you to write out all of the *arguments* in a single list and then the hardwork is done by the algorithm of preparing it into the normal *pipeline* format.

In [21]:
turb.pipe([["apply_columns","lower"],["apply_index","lower"]])

[('apply_columns', ('lower',), {}), ('apply_index', ('lower',), {})]

Keywords can be written as string and then automatically converted into objects/types downstream. For example the keyword `"axis=0"` in string converts the `"0"` into `0` integer.

Note that this shorthand does NOT accept non-basic arguments as keywords. For example, `lambda` expressions, functions and `dtypes` cannot be passed as keyword arguments. However strings, ints, booleans and floats are accepted.

In [22]:
turb.pipe([["apply_columns", "lower", "axis=0"]])

[('apply_columns', ('lower',), {'axis': 0})]

### Example: Using the operations above

Rather than setting `mode=delay`, we can simply create a pipe of the operations and then call `compute()` to either create a copy or operate inplace.

Here we will create several pipelines:

1. Pipeline one: expand prot IDs and drop duplicates, this is to *clean the labels*
2. Pipeline two: clean, groupby counters and gene names, and rename the columns ready for display.

In [85]:
g2 = turb.read("translation.csv", name="Translation")
g2

MetaPanda(Translation(n=5216, p=14, mem=0.585MB, key='None'), mode='instant')

#### Pipe 1

- uses MetaPanda `expand` function to split on `;` characters
- transforms that column to drop `"-"` end
- filter rows to keep all non-duplicates

#### Pipe 2

- Drops protein name
- Groupby the counter, Gene_names and calculate the mean
- Rename column information

Note that normally `groupby` and `mean` are separate functions in `pandas`, but in TurboPanda you can specify a built-in aggregator (such as `sum`, `mean`, `std`, `min`, `max`) by separating `groupby` with a double-underscore `__`:

Examples:

* `"groupby__mean"`, `"groupby__std"`, `"groupby__sum"`, `"groupby__count"`

In [90]:
pipe1 = turb.pipe([
    # expand on protein IDs
    ["expand", "prot_IDs", "sep=;"],
    # split of the -2,-3 etc to get duplicates
    ["transform", lambda x: x.str.split("-",expand=True)[0], "^prot_ID"],
    # filter out duplicates
    ["filter_rows", lambda x: ~x.duplicated(), "^prot_ID"],
])

pipe2 = turb.pipe([
    # drop protein name
    ["drop", "prot_name"],
    # groupby counter,gene names and then aggregate on mean
    ["apply", "groupby__mean", ["counter","Gene_names"]],
    # rename
    ["rename", [("translation","trans")]]
])

### Using `multi_compute()`

This provides yet another shorthand for executing multiple pipelines in a row.

In [94]:
g2_refreshed = g2.multi_compute([pipe1, pipe2])
g2_refreshed

MetaPanda(Translation(n=5190, p=11, mem=0.561MB, key='None'), mode='instant')

In [95]:
g2_refreshed.head()

,colnames,trans_G1_1,trans_G1_2,trans_G1_3,trans_G2M_1,trans_G2M_2,trans_G2M_3,trans_MG1_1,trans_MG1_2,trans_MG1_3,trans_S_1,trans_S_2
counter,Gene_names,,,,,,,,,,,
0,44M2.3,21.26058,20.47467,20.48905,21.01794,20.14569,22.29011,21.11775,20.71892,20.25788,20.58628,20.27662
1,A2M,22.62015,22.26825,23.11786,24.94606,24.21645,25.26399,23.56139,23.46051,22.21951,22.87688,23.35703
2,A2ML1,NaN,NaN,NaN,NaN,NaN,25.11629,NaN,NaN,NaN,NaN,NaN
3,AAAS,25.48382,24.42746,25.22645,24.44556,23.93706,25.30966,25.61462,25.45923,24.48253,24.31645,23.92143
4,AACS,24.18177,24.51533,24.32766,24.15993,24.05001,24.95797,24.11656,24.22523,23.96446,23.89440,23.78107
